 # Práctica  5.
 ----
  
  Universidad : UTEC \\
  Curso       : Inteligencia Artificial \\
  Profesor    : Cristian López Del Alamo \\
  Tema        : Árboles de Decisión \\
  

 ----

 Nombres y  Apellidos de Integrantes: (No olvide poner el % de participacion)
 - Integrante 1: Maria Fernanda Surco Vergara (100%)
 - Integrante 2: Brhandon Gutierrez Soto (100%)
 - Integrante 3:
 - Integrante 4:

*Una vez concluya la práctica debe subir el link de su colab a este  [Drive]
(https://docs.google.com/spreadsheets/d/1XCxGVmf8g29C7RZSPOqxvZHjjRZg45LHWR8mkZOfS1o/edit?usp=drivesdk)*


Para este ejercicio, usted deberá contruir su propio árbol de desición.
- Tenga en cuenta que se trata de un algoritmo recursivo.
- El caso base se da cuanto todos los elementos de un nodo tiene las mismas etiquetas, es decir, es un nodo terminal. Luego, el label de ese nodo toma el valor de la etiqueta común.
- En el caso que no sea un nodo terminal, el algoritmo debe buscar uno de los
feactures por el cual dividirse y para esto use Ganacia de Información (Entropía o Gini).
- Divida el dataset usando el feacture que genere una mayor ganancia de información en el padre o un menor GINI y llame recursivamente a la función create_DT.

Usted usará la base de datos iris, con 4 características y 3 clases.
Tome aleatoriamente 80% de los datos para crear el árbol y el resto para
probar el accuracy de la predicción.
Finalmente, muestra mediante una matriz de confusión el **accuracy** de su modelo.

Trabaje en equipo:

[Link de apoyo 1](https://towardsdatascience.com/the-simple-math-behind-3-decision-tree-splitting-criterions-85d4de2a75fe)

[Link de apoyo 2](https://www.quantstart.com/articles/Beginners-Guide-to-Decision-Trees-for-Supervised-Machine-Learning/)



In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

iris = sns.load_dataset('iris')
print(iris.head())




class Nodo:
 # Defina cuales será sus mimbros datos
  def __init__(self, X, Y, index):
        self.X = X  # Características
        self.Y = Y  # Tipo
        self.index = index  # Índice de las muestras en este nodo
        self.label = None  # Etiqueta del nodo (se establecerá si es un nodo terminal)
        self.feature_index = None  # Índice de la característica de división
        self.threshold = None  # Umbral de división
        self.child_nodes = {}  # Hijos del nodo

  def IsTerminal(self,Y):
    # retur true if this node have the sames labeles in Y
    return len(np.unique(Y)) == 1

  def entropy(self, Y):
        _, counts = np.unique(Y, return_counts=True)
        probabilities = counts / len(Y)
        entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))
        return entropy

  def information_gain(self, Y, left_mask, right_mask):
        # Calcula la ganancia de información
        parent_entropy = self.entropy(Y)
        left_entropy = self.entropy(Y[left_mask])
        right_entropy = self.entropy(Y[right_mask])
        total_samples = len(Y)

        info_gain = parent_entropy - (
            (len(Y[left_mask]) / total_samples) * left_entropy +
            (len(Y[right_mask]) / total_samples) * right_entropy
        )

        return info_gain

  def BestSplit(self,X,Y):
    # write your code here
    best_feature_index = None
    best_threshold = None
    best_info_gain = -1

    for x_index in range( X.shape[1]):
      val_unique= np.unique(X[:,x_index])
      for i in range(len(val_unique)-1):
        threshold = (val_unique[i] + val_unique[i + 1]) / 2
        left_mask = X[:, x_index] <= threshold
        right_mask = ~left_mask
        info_gain = self.information_gain(Y, left_mask, right_mask)
        if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_feature_index = x_index
                best_threshold = threshold

    return best_feature_index, best_threshold





class DT:
    def __init__(self, X, Y, index):
        self.X = X
        self.Y = Y
        self.index = index
        self.m_Root = None  # Inicializa m_Root como None
        self.create_DT(self.X, self.Y, [index] * len(Y))  # Pasa una lista de índices iguales

    def create_DT(self, X, Y, index):
      # Verifica si el nodo actual es terminal
      if Nodo(X,Y,index).IsTerminal(Y) == 1:
          most_common_label = np.unique(Y)[0]  # Get the most common label
          self.m_Root = Nodo(X, Y, index)  # Create a terminal node
          self.m_Root.label = most_common_label  # Assign the most common label to the terminal node
      else:
          self.m_Root = Nodo(X, Y, index)
          feature, threshold = self.m_Root.BestSplit(X, Y)  # Encuentra la mejor división
          self.m_Root.feature_index = feature  # Establece el índice de la característica
          self.m_Root.threshold = threshold  # Establece el umbral de división
          X_left = X[X[:, feature] <= threshold]
          Y_left = Y[X[:, feature] <= threshold]
          index_left = [index[i] for i in range(len(index)) if X[i, feature] <= threshold]

          X_right = X[X[:, feature] > threshold]
          Y_right = Y[X[:, feature] > threshold]
          index_right = [index[i] for i in range(len(index)) if X[i, feature] > threshold]

          # Crea un nodo no terminal y asigna a self.m_Root
          self.m_Root.izquierda = DT(X_left, Y_left, index_left).m_Root
          self.m_Root.derecha = DT(X_right, Y_right, index_right).m_Root


    def predict(self, X):
        predictions = []
        for sample in X:
            label = self.predict_sample(sample, self.m_Root)
            predictions.append(label)
        return predictions

    def predict_sample(self, sample, node):
        if node.label is not None:
            return node.label

        feature_index, threshold = node.feature_index, node.threshold
        if threshold is not None:
            if sample[feature_index] <= threshold:
                return self.predict_sample(sample, node.izquierda)
            else:
                return self.predict_sample(sample, node.derecha)
        else:

            return node.label


X_train, X_test, y_train, y_test = train_test_split(iris.drop('species', axis=1), iris['species'], test_size=0.2, random_state=42)

# Crear una instancia de DT con los datos de entrenamiento
dt = DT(X_train.values, y_train.values, index=0)  # Cambia index a un valor único, en este caso, 0

# Realizar predicciones en el conjunto de prueba
y_pred = dt.predict(X_test.values)

# Calcula la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Imprime la matriz de confusión
print(cm)

accuracy = accuracy_score(y_test.values, y_pred)

print(f"Exactitud: {accuracy}")

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
Exactitud: 1.0
